In [28]:
import numpy as np
from scipy.optimize import least_squares

# 震动波的传播速度 (m/s)
speed_of_sound = 340

# 监测设备的三维坐标 (经度, 纬度, 高度) 和音爆到达时间 (单位: 秒)
devices = np.array([
    [110.241, 27.204, 824, 100.767],
    [110.780, 27.456, 727, 112.220],
    [110.712, 27.785, 742, 188.020],
    [110.251, 27.825, 850, 258.985],
    [110.524, 27.617, 786, 118.443],
    [110.467, 27.921, 678, 266.871],
    [110.047, 27.121, 575, 163.024]
])

# 经纬度转换为米，纬度间每度 111263 米，经度间每度 97304 米
lat_conversion = 111263
lon_conversion = 97304

# 转换经纬度为米并计算距离
device_positions = np.array([
    [lon_conversion * (device[0] - devices[0, 0]), 
     lat_conversion * (device[1] - devices[0, 1]), 
     device[2]]
    for device in devices
])

# 音爆到达时间
arrival_times = devices[:, 3]

# 定义损失函数，用于最小化距离的差异
def residuals(params, selected_devices, selected_times):
    x, y, z, t0 = params
    theoretical_times = [
        t0 + np.linalg.norm([x - pos[0], y - pos[1], z - pos[2]]) / speed_of_sound
        for pos in selected_devices
    ]
    return np.array(theoretical_times) - selected_times

# 定义一个函数来测试不同数量的设备并添加约束条件
def test_min_devices(min_devices):
    for i in range(min_devices, len(devices) + 1):
        # 随机选择 i 台设备
        indices = np.random.choice(len(devices), i, replace=False)
        selected_devices = device_positions[indices]
        selected_times = arrival_times[indices]

        # 初始猜测
        initial_guess = np.array([0, 0, 150, 4.00])

        # 设置约束条件：t0 >= 0
        bounds = ([-np.inf, -np.inf, -np.inf, 0], np.inf)

        # 使用最小二乘法，带有约束条件
        result = least_squares(residuals, initial_guess, bounds=bounds, args=(selected_devices, selected_times))

        # 打印结果
        print(f"使用 {i} 台设备时的残骸位置和时间估计:")
        print(f"残骸的音爆发生位置: (x, y, z) = {result.x[:3]} 米")
        print(f"音爆发生的时间: t0 = {result.x[3]} 秒")
        print("--------------------------------------------------")

# 从3台设备开始测试，直到7台设备
test_min_devices(3)


使用 3 台设备时的残骸位置和时间估计:
残骸的音爆发生位置: (x, y, z) = [25168.16478614 12447.39591772 -7500.0744749 ] 米
音爆发生的时间: t0 = 16.693446637305232 秒
--------------------------------------------------
使用 4 台设备时的残骸位置和时间估计:
残骸的音爆发生位置: (x, y, z) = [35481.61232037 -7947.60934302   895.88951714] 米
音爆发生的时间: t0 = 1.5268282732257082e-20 秒
--------------------------------------------------
使用 5 台设备时的残骸位置和时间估计:
残骸的音爆发生位置: (x, y, z) = [35937.26594839 -9096.10973694   794.699741  ] 米
音爆发生的时间: t0 = 7.765263286733346e-16 秒
--------------------------------------------------
使用 6 台设备时的残骸位置和时间估计:
残骸的音爆发生位置: (x, y, z) = [35375.1442562  -1406.77420178   761.63434684] 米
音爆发生的时间: t0 = 1.235320615345954 秒
--------------------------------------------------
使用 7 台设备时的残骸位置和时间估计:
残骸的音爆发生位置: (x, y, z) = [36181.8261211  -3573.81891345   849.1663673 ] 米
音爆发生的时间: t0 = 8.941905891859472e-21 秒
--------------------------------------------------


In [29]:
import numpy as np
from scipy.optimize import least_squares

# 震动波的传播速度 (m/s)
speed_of_sound = 340

# 监测设备的三维坐标 (经度, 纬度, 高度) 和音爆到达时间 (单位: 秒)
devices = np.array([
    [110.241, 27.204, 824, 100.767],
    [110.780, 27.456, 727, 112.220],
    [110.712, 27.785, 742, 188.020],
    [110.251, 27.825, 850, 258.985],
    [110.524, 27.617, 786, 118.443],
    [110.467, 27.921, 678, 266.871],
    [110.047, 27.121, 575, 163.024]
])

# 经纬度转换为米，纬度间每度 111263 米，经度间每度 97304 米
lat_conversion = 111263
lon_conversion = 97304

# 转换经纬度为米并计算距离
device_positions = np.array([
    [lon_conversion * (device[0] - devices[0, 0]), 
     lat_conversion * (device[1] - devices[0, 1]), 
     device[2]]
    for device in devices
])

# 音爆到达时间
arrival_times = devices[:, 3]

# 定义残骸的真实位置（假设值，或可根据某次实际结果得到）
true_position = np.array([1000, 1000, 500])
true_time = 200.0

# 定义损失函数，用于最小化距离的差异
def residuals(params, selected_devices, selected_times):
    x, y, z, t0 = params
    theoretical_times = [
        t0 + np.linalg.norm([x - pos[0], y - pos[1], z - pos[2]]) / speed_of_sound
        for pos in selected_devices
    ]
    return np.array(theoretical_times) - selected_times

# 定义一个函数来测试不同数量的设备并量化其稳定性
def test_stability(min_devices, max_experiments=100):
    stability_results = []
    
    for i in range(min_devices, len(devices) + 1):
        results = []
        
        for _ in range(max_experiments):
            # 随机选择 i 台设备
            indices = np.random.choice(len(devices), i, replace=False)
            selected_devices = device_positions[indices]
            selected_times = arrival_times[indices]

            # 初始猜测
            initial_guess = np.array([0, 0, 150, 4.00])

            # 设置约束条件：t0 >= 0
            bounds = ([-np.inf, -np.inf, -np.inf, 0], np.inf)

            # 使用最小二乘法，带有约束条件
            result = least_squares(residuals, initial_guess, bounds=bounds, args=(selected_devices, selected_times))

            # 记录预测结果
            predicted_position = result.x[:3]
            predicted_time = result.x[3]
            results.append(np.concatenate([predicted_position, [predicted_time]]))
        
        # 计算每次实验结果的均值和标准差，量化稳定性
        results = np.array(results)
        mean_result = np.mean(results, axis=0)
        std_deviation = np.std(results, axis=0)
        
        # 记录当前设备数量下的稳定性指标
        stability_results.append((i, mean_result, std_deviation))
        print(f"设备数量: {i}, 平均结果: {mean_result}, 标准差: {std_deviation}")
        
    return stability_results

# 从3台设备开始测试，最多进行100次实验以评估稳定性
stability_data = test_stability(3, max_experiments=100)


设备数量: 3, 平均结果: [ 3.60068973e+04  4.65262842e+03 -1.90467058e+03  1.81983083e+01], 标准差: [15339.0089405  13781.5942725   7179.36929903    33.95952961]
设备数量: 4, 平均结果: [ 3.48382789e+04  1.23138036e+03 -9.93100364e+01  9.91790758e+00], 标准差: [6457.69589713 9043.536792   5222.21576424   17.51652891]
设备数量: 5, 平均结果: [ 3.62264166e+04 -9.35974070e+02  7.95888772e+02  7.55066673e+00], 标准差: [2662.66439826 8364.57197914  165.79887435   20.08686059]
设备数量: 6, 平均结果: [ 3.57524742e+04 -3.20784827e+03  8.37914640e+02  1.51260323e+00], 标准差: [1860.63031073 2606.46649834   85.26425715    3.48008336]
设备数量: 7, 平均结果: [ 3.61818261e+04 -3.57381922e+03  8.49158427e+02  8.58119456e-21], 标准差: [1.10461591e-04 6.00110775e-04 4.25061126e-02 2.00703960e-21]


In [32]:
import numpy as np
from scipy.optimize import least_squares

# 震动波的传播速度 (m/s)
speed_of_sound = 340

# 监测设备的三维坐标 (经度, 纬度, 高度) 和音爆到达时间 (单位: 秒)
devices = np.array([
    [110.241, 27.204, 824, 100.767],
    [110.780, 27.456, 727, 112.220],
    [110.712, 27.785, 742, 188.020],
    [110.251, 27.825, 850, 258.985],
    [110.524, 27.617, 786, 118.443],
    [110.467, 27.921, 678, 266.871],
    [110.047, 27.121, 575, 163.024]
])

# 经纬度转换为米，纬度间每度 111263 米，经度间每度 97304 米
lat_conversion = 111263
lon_conversion = 97304

# 转换经纬度为米并计算距离
device_positions = np.array([
    [lon_conversion * (device[0] - devices[0, 0]), 
     lat_conversion * (device[1] - devices[0, 1]), 
     device[2]]
    for device in devices
])

# 音爆到达时间
arrival_times = devices[:, 3]

# 定义残骸位置和音爆时间的未知量
# 初始猜测：假设残骸发生音爆的初始位置在设备的第一个位置附近，音爆时间接近第一个设备的到达时间减去传播时间。
initial_guess = np.array([0, 0, 150, 4.00])

# 定义损失函数，用于最小化距离的差异
def residuals(params):
    # 提取残骸位置和时间
    x, y, z, t0 = params
    
    # 计算音爆到每个设备的理论时间
    theoretical_times = [
        t0 + np.linalg.norm([x - pos[0], y - pos[1], z - pos[2]]) / speed_of_sound
        for pos in device_positions
    ]
    
    # 返回实际到达时间与理论到达时间之间的差
    return np.array(theoretical_times) - arrival_times

# 使用信赖域反射算法（trf）进行最小二乘法求解非线性方程组，限制 t0 > 0
lower_bounds = [-np.inf, -np.inf, -np.inf, 0]  # 对 t0 设置下限为 0
upper_bounds = [np.inf, np.inf, np.inf, np.inf]  # 不限制其他参数的上限

# 使用 least_squares 进行带有边界约束的最小二乘法求解
result = least_squares(residuals, initial_guess, bounds=(lower_bounds, upper_bounds), method='trf')

# 输出结果
print(f"残骸的音爆发生位置: (x, y, z) = {result.x[:3]} 米")
print(f"音爆发生的时间: t0 = {result.x[3]} 秒")

# 提取优化结果
x, y, z, t0 = result.x

# 转换回经纬度，基于设备1的经纬度位置作为基准
reference_lon, reference_lat = devices[0, 0], devices[0, 1]

# 经度转换回度数
longitude = x / lon_conversion + reference_lon

# 纬度转换回度数
latitude = y / lat_conversion + reference_lat

# 输出残骸的音爆位置和时间
print(f"残骸的音爆发生位置 (经度, 纬度, 高度): ({longitude}, {latitude}, {z} 米)")
print(f"音爆发生的时间 t0 = {t0} 秒")


残骸的音爆发生位置: (x, y, z) = [36181.82620437 -3573.8203321    849.1392796 ] 米
音爆发生的时间: t0 = 5.91172975258385e-21 秒
残骸的音爆发生位置 (经度, 纬度, 高度): (110.61284315346103, 27.171879525699502, 849.1392796009543 米)
音爆发生的时间 t0 = 5.91172975258385e-21 秒


In [21]:
import numpy as np
from scipy.optimize import least_squares

# 震动波的传播速度 (m/s)
speed_of_sound = 340

# 监测设备的三维坐标 (经度, 纬度, 高度) 和音爆到达时间 (单位: 秒)
devices = np.array([
    [110.241, 27.204, 824, 100.767],
    [110.780, 27.456, 727, 112.220],
    [110.712, 27.785, 742, 188.020],
    [110.251, 27.825, 850, 258.985],
    [110.524, 27.617, 786, 118.443],
    [110.467, 27.921, 678, 266.871],
    [110.047, 27.121, 575, 163.024]
])

# 经纬度转换为米，纬度间每度 111263 米，经度间每度 97304 米
lat_conversion = 111263
lon_conversion = 97304

# 转换经纬度为米并计算距离
device_positions = np.array([
    [lon_conversion * (device[0] - devices[0, 0]), 
     lat_conversion * (device[1] - devices[0, 1]), 
     device[2]]
    for device in devices
])

# 音爆到达时间
arrival_times = devices[:, 3]

# 定义残骸位置和音爆时间的未知量
# 初始猜测：假设残骸发生音爆的初始位置在设备的第一个位置附近，音爆时间接近第一个设备的到达时间减去传播时间。
initial_guess = np.array([0, 0, 150, 4.00])

# 定义损失函数，用于最小化距离的差异
def residuals(params):
    # 提取残骸位置和时间
    x, y, z, t0 = params
    
    # 计算音爆到每个设备的理论时间
    theoretical_times = [
        t0 + np.linalg.norm([x - pos[0], y - pos[1], z - pos[2]]) / speed_of_sound
        for pos in device_positions
    ]
    
    # 返回实际到达时间与理论到达时间之间的差
    return np.array(theoretical_times) - arrival_times

# 使用信赖域反射算法（trf）进行最小二乘法求解非线性方程组，限制 t0 > 0
lower_bounds = [-np.inf, -np.inf, -np.inf, 0]  # 对 t0 设置下限为 0
upper_bounds = [np.inf, np.inf, np.inf, np.inf]  # 不限制其他参数的上限

# 使用 least_squares 进行带有边界约束的最小二乘法求解
result = least_squares(residuals, initial_guess, bounds=(lower_bounds, upper_bounds), method='trf', verbose=2)

# 输出详细的优化结果
print("\n所有解的值：")
print(f"残骸的音爆发生位置 (x, y, z): {result.x[:3]} 米")
print(f"音爆发生的时间 t0: {result.x[3]} 秒")
print(f"迭代次数: {result.nfev}")
print(f"优化终止原因: {result.message}")


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.5102e+04                                    1.04e+02    
       1              2         1.4206e+04      8.96e+02       1.50e+02       6.55e-01    
       2              3         1.3955e+04      2.51e+02       3.00e+02       2.32e+00    
       3              4         1.3429e+04      5.26e+02       6.00e+02       4.55e+00    
       4              5         1.2373e+04      1.06e+03       1.20e+03       6.33e+00    
       5              6         1.0390e+04      1.98e+03       2.40e+03       9.85e+00    
       6              7         7.0746e+03      3.32e+03       4.80e+03       3.17e+01    
       7              8         2.9456e+03      4.13e+03       9.60e+03       2.34e+02    
       8              9         1.2773e+03      1.67e+03       1.92e+04       8.21e+02    
       9             11         1.1265e+03      1.51e+02       9.60e+03       3.09e+02    

In [24]:
import numpy as np
from scipy.optimize import minimize
import itertools

# 假设表1中的数据：7个设备的坐标
coordinates = np.array([
    [110.241,27.204,824,100.767],
    [110.780,27.456,727,112.220],
    [110.712,27.785,742,188.020],
    [110.251,27.825,850,258.985],
    [110.524,27.617,786,118.443],
    [110.467,27.921,678,266.871],
    [110.047,27.121,575,163.024]
])

# 对应的音爆到达时间
arrival_times = np.array([100.767,112.220,188.020,258.985,118.443,266.871,163.024])

# 震动波的传播速度
sound_speed = 340  # m/s

# 定义目标函数用于最优化位置
def error_function(x, coords, times):
    predicted_times = np.linalg.norm(coords - x, axis=1) / sound_speed
    return np.sum((predicted_times - times) ** 2)

# 选择贡献度最高的设备集
def select_best_devices(coordinates, arrival_times, num_devices_to_select=6):
    best_combination = None
    best_error = float('inf')
    best_position = None
    
    # 从7台设备中选取组合
    for subset in itertools.combinations(range(len(coordinates)), num_devices_to_select):
        subset_coords = coordinates[list(subset)]
        subset_times = arrival_times[list(subset)]
        
        # 优化残骸位置
        initial_guess = np.mean(subset_coords, axis=0)
        result = minimize(error_function, initial_guess, args=(subset_coords, subset_times))
        
        # 记录误差最小的设备组合
        if result.fun < best_error:
            best_error = result.fun
            best_combination = subset
            best_position = result.x
    
    return best_combination, best_position

# 假设我们希望选择贡献度最高的6个设备
best_devices, predicted_position = select_best_devices(coordinates, arrival_times, num_devices_to_select=6)

print("最合适的设备组合为：", best_devices)
print("预测的音爆位置为：", predicted_position)


最合适的设备组合为： (0, 1, 2, 3, 4, 6)
预测的音爆位置为： [  -619.08305074   -702.00755074 -26727.49947022 -45558.97726325]


In [31]:
import numpy as np
from scipy.optimize import least_squares
# 震动波的传播速度 (m/s)
speed_of_sound = 340
# 监测设备的三维坐标 (经度, 纬度, 高度) 和音爆到达时间 (单位: 秒)
devices = np.array([
    [110.241,27.204,824,100.767],
    [110.780,27.456,727,112.220],
    [110.712,27.785,742,188.020],
    [110.251,27.825,850,258.985],
    [110.524,27.617,786,118.443],
    [110.467,27.921,678,266.871],
    [110.047,27.121,575,163.024]
])
# 经纬度转换为米，纬度间每度 111263 米，经度间每度 97304 米
lat_conversion = 111263
lon_conversion = 97304
# 转换经纬度为米并计算距离
device_positions = np.array([
    [lon_conversion * (device[0] - devices[0, 0]), 
     lat_conversion * (device[1] - devices[0, 1]), 
     device[2]]
    for device in devices
])

# 音爆到达时间
arrival_times = devices[:, 3]
arrival_times=devices[:,3]
initial_guess = np.array([36181.82620437 ,-3573.8203321  ,  849.1392796,5.91172975258385e-21])
x,y,z,t0=initial_guess
p=[np.linalg.norm([x - pos[0], y - pos[1], z - pos[2]]) / speed_of_sound
                         for pos in device_positions]
# 计算理论时间与实际时间的差值（残差）
residuals = np.array(p) - arrival_times

# 输出每个设备的残差
print("每个设备的残差: ", residuals)

# 计算平均误差和标准差
mean_error = np.mean(residuals)
std_error = np.std(residuals)

print(f"平均误差: {mean_error:.4f} 秒")
print(f"标准差: {std_error:.4f} 秒")

每个设备的残差:  [  6.16801937  -7.65752605  14.61716402 -21.48945857  29.42259859
 -18.19738584  -0.23064298]
平均误差: 0.3761 秒
标准差: 16.7675 秒
